In [1]:
from Bio import Entrez
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import time
import re
import os
import pickle

Entrez.email = "something@something.com"    
directory = os.getcwd() # setting working directory

In [2]:
def get_key(d, value):  # a module to look throught a dictionary values
    for k, v in d.items():
        if v == value:
            return k

def save_obj(obj, name ):   # we have to store big dictionaries that take a lot of machine time
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


def load_obj(name):  # and load them
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)
    
len_fasta_out_file = 60
def module_fasta_writer(out_file, line):  # this module writes in fasta format by 60 nums in each line
    start_point = 0
    end_point = len_fasta_out_file
    if len(line) > len_fasta_out_file:
        len_line = len(line)
        while len_line > 0:
            j = line[start_point:end_point]
            len_line = len_line - len_fasta_out_file
            out_file.write(j + "\n")
            start_point = start_point + len_fasta_out_file
            end_point = end_point + len_fasta_out_file
    else:
        out_file.write(line + "\n")

In [49]:
gene_to_find = "CO1[gene]" # setting the gene of the interest 1

In [ ]:
CYTB
CO1
16S

In [25]:
handle = Entrez.esearch(db = "nucleotide", term = gene_to_find, RetMax = 100000)  # making a handle of ids
record = Entrez.read(handle)
#print("Seq names:", record["IdList"])
#print("")
#print("Total:", record["Count"])
gi_list = record["IdList"]  # making a list from the handle

In [26]:
gi_list = record["IdList"]
handle = Entrez.efetch(db="nucleotide", id=gi_list, rettype="gb", retmode="text")

dict_of_records = {}
fafile = open(directory + "/collected_records_" + gene_to_find + ".fasta", 'w')
fafile.write("ID" + "\t" + "source" + "\t" + "organism" + "\n")

for seq_record in SeqIO.parse(handle, "genbank"):  # looking for sequencies
    description = seq_record.annotations["source"] + "_" + seq_record.annotations["organism"] + "_" +  seq_record.annotations["date"]
    #print(seq_record.id)
    dict_of_records[seq_record.id] = [description, seq_record.seq] # loading them into the dictionary
    #print(record.format("fasta")) 
    #print(seq_record.annotations["references"][1])
    #print(seq_record.annotations["date"])
    #print(seq_record.annotations)
    #print(seq_record.seq)
    line = seq_record.id + "\t" + seq_record.annotations["source"] + "\t" + seq_record.annotations["organism"] + "\n"
    #print(line)
    fafile.write(line)
fafile.close()


In [30]:
save_obj(dict_of_records, gene_to_find)  # you can just back up your dictionary

In [50]:
dict_of_records = load_obj(gene_to_find)  # or refresh it
#print(dict_of_records)

Usefull NCBI request script 
# This assumes you have already run a search as shown above,
# and set the variables count, webenv, query_key
gi_list = record["IdList"]
count = len(gi_list)

try:
    from urllib.error import HTTPError  # for Python 3
except ImportError:
    from urllib2 import HTTPError  # for Python 2

batch_size = 3
out_handle = open("out.fasta", "w")
for start in range(0, count, batch_size):
    end = min(count, start+batch_size)
    print("Going to download record %i to %i" % (start+1, end))
    attempt = 0
    while attempt < 3:
        attempt += 1
        try:
            fetch_handle = Entrez.efetch(db="nucleotide", id=gi_list, rettype="gb", retmode="text", retstart=start, retmax=batch_size,
                                         idtype="acc")

        except HTTPError as err:
            if 500 <= err.code <= 599:
                print("Received error from server %s" % err)
                print("Attempt %i of 3" % attempt)
                time.sleep(15)
            else:
                raise
    data = fetch_handle.read()
    fetch_handle.close()
    out_handle.write(data)
out_handle.close()



In [52]:
gene_to_find_2 = "16S[gene]"  # setting the gene of the interest 2

In [53]:
handle_2 = Entrez.esearch(db = "nucleotide", term = gene_to_find_2, RetMax = 100000)
record_2 = Entrez.read(handle_2)
#rint("Seq names:", record_2["IdList"])
#print("")
#print("Total:", record_2["Count"])
gi_list_2 = record_2["IdList"]

In [63]:
gi_list_2 = record_2["IdList"]
handle_2 = Entrez.efetch(db="nucleotide", id=gi_list_2, rettype="gb", retmode="text")

dict_of_records_2 = {}
fafile_2 = open(directory + "/collected_records_" + gene_to_find_2 + ".fasta", 'w')
fafile_2.write("ID" + "\t" + "source" + "\t" + "organism"  + "\n")

for seq_record in SeqIO.parse(handle_2, "genbank"):
    description = seq_record.annotations["source"] + "_" + seq_record.annotations["organism"] + "_" +  seq_record.annotations["date"]
    #print(seq_record.id)
    dict_of_records_2[seq_record.id] = [description, seq_record.seq]
    #print(record.format("fasta")) 
    #print(seq_record.annotations["references"])
    #print(seq_record.annotations)
    #print(seq_record.seq)
    line = seq_record.id + "\t" + seq_record.annotations["source"] + "\t" + seq_record.annotations["organism"] + "\n"
    #print(line)
    fafile_2.write(line)
fafile_2.close()


In [31]:
save_obj(dict_of_records_2, gene_to_find_2)

In [47]:
dict_of_records_2 = load_obj(gene_to_find_2)
#print(dict_of_records_2)

In [36]:
intersection_list = list(set([x[0].split("_")[1] for x in dict_of_records_2.values()]).intersection([x[0].split("_")[1] for x in dict_of_records.values()]))
print(intersection_list)  # we can calculate the intersection of two genes in findings

['Batrachuperus pinchonii', 'Pseudohynobius puxiongensis', 'Galba truncatula']


In [8]:
intersection_list_2 = list(set([x for x in dict_of_records_2.keys()]).intersection([x for x in dict_of_records.keys()]))
print(intersection_list_2)  # a negative control

[]


In [51]:
fafile_out = open(directory + "/collected_records_" + gene_to_find + "_" + gene_to_find_2 + ".fasta", 'w')
fafile_out.write("ID" + "\t" + "gene" + "\t" + "organism"  + "\n")
      
intersection_list = list(set([x[0].split("_")[1] for x in dict_of_records_2.values()]).intersection([x[0].split("_")[1] for x in dict_of_records.values()]))
#print(intersection_list)
zipped = zip([x[0].split("_")[1] for x in dict_of_records.values()], dict_of_records.keys())
zipped_2 = zip([x[0].split("_")[1] for x in dict_of_records_2.values()], dict_of_records_2.keys())

#!rm files
#!mkdir ./files
!cd ./files
#!pwd


for i, j in zipped:
    #print(i)
    if i in intersection_list:  # selecting records and writing them into merge files and log files
        with open("./files/" + str(i) + "_" + gene_to_find + '_merged.fasta', 'a') as f:
            f.write(">" + str(j) + " " + str(i) +  "\n")
            module_fasta_writer(f, str(dict_of_records[j][1]))
        fafile_out.write("\n")
        fafile_out.write(j + "\t" + "16S" + "\t" + i  + "\n")
        
        #print("OK")
        
for i, j in zipped_2:
    if i in intersection_list:
        with open("./files/" + str(i) + "_" + gene_to_find_2 + '_merged.fasta', 'a') as f:
            f.write(">" + str(j) + " " + str(i) +  "\n")
            module_fasta_writer(f, str(dict_of_records_2[j][1]))
        fafile_out.write("\n")
        fafile_out.write(j + "\t" + "CO1" + "\t" + i  + "\n")
        #print("OK")
print("Done")

Done
